# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# !jupyter nbextension enable --py --sys-prefix widgetsnbextension

# jupyter labextension install @jupyter-widgets/jupyterlab-manager
# !pip install gmaps
# !jupyter nbextension enable --py --sys-prefix gmaps

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Disable unnecessary SettingWithCopyWarning in Pandas
pd.options.mode.chained_assignment = None  # default='warn'

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the csv exported in Part I to a DataFrame
cities_weather_data_path = "../output_data/cities.csv"
cities_data = pd.read_csv(cities_weather_data_path)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations and Humidity as the weight
locations = cities_data[["Lat", "Lng"]]
humidity = cities_data["Humidity"]

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Narrow down the cities to fit weather conditions.
    # A max temperature lower than 80 degrees but higher than 70.
    # Wind speed less than 10 mph.
    # Zero cloudiness.
cities_narrowed = cities_data.loc[(cities_data["Max Temp"] <= 100) &
                                  (cities_data["Max Temp"] >= 50) &
                                  (cities_data["Wind Speed"] <= 30) &
                                  (cities_data["Cloudiness"] == 0)]

# Drop any rows that don't contain all three conditions.
cities_narrowed

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,busselton,-33.6500,115.3333,58.75,80,0,10.18,AU,1639919125
24,constitucion,-35.3333,-72.4167,68.50,52,0,5.79,CL,1639919184
57,guerrero negro,27.9769,-114.0611,52.45,44,0,4.05,MX,1639919356
61,lindi,-10.0000,39.7167,88.38,57,0,10.27,TZ,1639919218
68,constantine,36.3650,6.6147,58.51,37,0,2.04,DZ,1639919117
69,trelew,-43.2490,-65.3051,81.43,32,0,17.85,AR,1639919360
79,rawson,-43.3002,-65.1023,78.40,38,0,15.59,AR,1639919362
94,libourne,44.9167,-0.2500,50.56,63,0,4.74,FR,1639919396
95,salon,26.0333,81.4500,56.80,33,0,7.67,IN,1639919397
121,pasighat,28.0667,95.3333,61.09,49,0,4.54,IN,1639919403


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Store into variable named hotel_df
hotel_df = cities_narrowed

# Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""

In [6]:
# Set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Hit the Google Places API for each city's coordinates, and store the first Hotel result into the DataFrame.

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get city from df for hotel
    hotel_city = row['City']

    # add keyword and location coordinates to params dict
    params['keyword'] = hotel_city
    params['location'] = f"{row['Lat']},{row['Lng']}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {hotel_city}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest {hotel_city} hotel is {results[0]['name']}.")       
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 18: busselton.
Missing field/result... skipping.
------------
Retrieving Results for Index 24: constitucion.
Closest constitucion hotel is Nuevo Hotel Constitucion.
------------
Retrieving Results for Index 57: guerrero negro.
Missing field/result... skipping.
------------
Retrieving Results for Index 61: lindi.
Missing field/result... skipping.
------------
Retrieving Results for Index 68: constantine.
Closest constantine hotel is Novotel Constantine.
------------
Retrieving Results for Index 69: trelew.
Missing field/result... skipping.
------------
Retrieving Results for Index 79: rawson.
Missing field/result... skipping.
------------
Retrieving Results for Index 94: libourne.
Missing field/result... skipping.
------------
Retrieving Results for Index 95: salon.
Missing field/result... skipping.
------------
Retrieving Results for Index 121: pasighat.
Missing field/result... skipping.
------------
Retrieving Results for Index 160: geraldton.
Missing fiel

In [7]:
# Remove cities with no hotels found
final_hotel_df = hotel_df.copy()
final_hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
final_hotel_df.dropna(subset=['Hotel Name'], inplace=True)

In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in final_hotel_df.iterrows()]
locations = final_hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
            info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))